In [1]:
# Add rebel component https://github.com/Babelscape/rebel/blob/main/spacy_component.py
import spacy
import crosslingual_coreference
import requests
import re
import hashlib
import pandas as pd
from spacy import Language
from typing import List
from spacy.tokens import Doc, Span
from transformers import pipeline

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\GCM\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
def call_wiki_api(item):
    try:
        url = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&search={item}&language=en&format=json"
        data = requests.get(url).json()
        # Return the first id (Could upgrade this in the future)
        return data['search'][0]['id']
    except:
        return 'id-less'

def extract_triplets(text):
    """
    Function to parse the generated text and extract the triplets
    """
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})

    return triplets


@Language.factory(
    "rebel",
    requires=["doc.sents"],
    assigns=["doc._.rel"],
    default_config={
        "model_name": "Babelscape/rebel-large",
        "device": 0,
    },
)
class RebelComponent:
    def __init__(
        self,
        nlp,
        name,
        model_name: str,
        device: int,
    ):
        assert model_name is not None, ""
        self.triplet_extractor = pipeline("text2text-generation", model=model_name, tokenizer=model_name, device=device)
        self.entity_mapping = {}
        # Register custom extension on the Doc
        if not Doc.has_extension("rel"):
            Doc.set_extension("rel", default={})

    def get_wiki_id(self, item: str):
        mapping = self.entity_mapping.get(item)
        if mapping:
            return mapping
        else:
            res = call_wiki_api(item)
            self.entity_mapping[item] = res
            return res

    
    def _generate_triplets(self, sent: Span) -> List[dict]:
        output_ids = self.triplet_extractor(sent.text, return_tensors=True, return_text=False)[0]["generated_token_ids"]["output_ids"]
        extracted_text = self.triplet_extractor.tokenizer.batch_decode(output_ids[0])
        extracted_triplets = extract_triplets(extracted_text[0])
        return extracted_triplets

    def set_annotations(self, doc: Doc, triplets: List[dict]):
        for triplet in triplets:

            # Remove self-loops (relationships that start and end at the entity)
            if triplet['head'] == triplet['tail']:
                continue

            # Use regex to search for entities
            head_span = re.search(triplet["head"], doc.text)
            tail_span = re.search(triplet["tail"], doc.text)

            # Skip the relation if both head and tail entities are not present in the text
            # Sometimes the Rebel model hallucinates some entities
            if not head_span or not tail_span:
                continue

            index = hashlib.sha1("".join([triplet['head'], triplet['tail'], triplet['type']]).encode('utf-8')).hexdigest()
            if index not in doc._.rel:
                # Get wiki ids and store results
                doc._.rel[index] = {"relation": triplet["type"], "head_span": {'text': triplet['head'], 'id': self.get_wiki_id(triplet['head'])}, "tail_span": {'text': triplet['tail'], 'id': self.get_wiki_id(triplet['tail'])}}

    def __call__(self, doc: Doc) -> Doc:
        for sent in doc.sents:
            sentence_triplets = self._generate_triplets(sent)
            self.set_annotations(doc, sentence_triplets)
        return doc

In [3]:
DEVICE = 0 # Number of the GPU, -1 if want to use CPU

# Add coreference resolution model
coref = spacy.load('en_core_web_sm', disable=['ner', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer'])
coref.add_pipe("xx_coref", 
    config={"chunk_size": 2500, 
            "chunk_overlap": 2, 
            "device": DEVICE}
              )

# Define rel extraction model
rel_ext = spacy.load('en_core_web_sm', disable=['ner', 'lemmatizer', 'attribute_rules', 'tagger'])
rel_ext.add_pipe("rebel", 
                 config={
                     'device':DEVICE,
                     'model_name':'Babelscape/rebel-large'} # Model used, will default to 'Babelscape/rebel-large' if not given
                )

error loading _jsonnet (this is expected on Windows), treating C:\Users\GCM\AppData\Local\Temp\tmp5paoxsgv\config.json as plain json
Some weights of the model checkpoint at nreimers/mMiniLMv2-L12-H384-distilled-from-XLMR-Large were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaModel were not initialized from the model checkpoint at nreimers/mMiniLMv2-L12-H384-distilled-from-XLMR-Large and 

In [4]:
# crosslingual_coreference implementation
def coref_res(text_series):
    coref_text_series = text_series.apply(lambda x : coref(x)._.resolved_text)
    return(coref_text_series)

# # choose minilm for speed/memory and info_xlm for accuracy
# predictor = Predictor(
#     language="en_core_web_sm", device=-1, model_name="minilm"
# )

In [5]:
def link_entities_text(text):
    try:
        ent_rel_lst = list(rel_ext(text)._.rel.values())
    except:
        print("Could not extract relationships for text")
        ent_rel_lst = [{'relation': 'rel_err',
                        'head_span': {'text': 'rel_err', 'id': 'rel_err'},
                        'tail_span': {'text': 'rel_err', 'id': 'rel_err'}}]
        
    entity_df = pd.DataFrame()
    rel_lst = []
    head_text_lst = []
    head_wiki_id_lst = []
    tail_text_lst = []
    tail_wiki_id_lst = []
    for i in range(len(ent_rel_lst)):
        rel_lst.append(ent_rel_lst[i]['relation'])
        head_text_lst.append(ent_rel_lst[i]['head_span']['text'])
        head_wiki_id_lst.append(ent_rel_lst[i]['head_span']['id'])
        tail_text_lst.append(ent_rel_lst[i]['tail_span']['text'])
        tail_wiki_id_lst.append(ent_rel_lst[i]['tail_span']['id'])
    entity_df['head_text'] = head_text_lst
    entity_df['head_wiki_id'] = head_wiki_id_lst
    entity_df['relation'] = rel_lst
    entity_df['tail_text'] = tail_text_lst
    entity_df['tail_wiki_id'] = tail_wiki_id_lst
    return(entity_df)

def link_entities(text_series):
    entity_df_series = text_series.apply(lambda x : link_entities_text(x))
    return(entity_df_series)

In [7]:
data_df = pd.read_csv('E:\GIT_REPOS\LAB\Literature_summary\TPN\Papers\\scopus.csv')
data_df

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,ISBN,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID
0,"Nemir P., Jr., Hawthorne H.R., Lecrone B.L.",6701334034;24736115000;24736458400;,Simple Intestinal Obstruction:Prolongation of ...,1948,Proceedings of the Society for Experimental Bi...,69,1,NaN,14,16,...,NaN,NaN,18889765.0,English,Proc. Soc. Exp. Biol. Med.,Article,Final,NaN,Scopus,2-s2.0-84911326504
1,"FROST P.M., SMITH J.L.",35505269600;7410166466;,Influence of potassium salts on efficiency of ...,1953,Metabolism: clinical and experimental,2,6,NaN,529,535,...,NaN,NaN,13110761.0,English,Metab. Clin. Exp.,Article,Final,NaN,Scopus,2-s2.0-0013653192
2,"LEVINE R.S., CALVARY E.C., PLZAK J.E., ALLEN J.G.",26631202100;24545044500;57178904400;57178257500;,Effect of parenterally administered fat emulsi...,1957,Metabolism: clinical and experimental,6,6,NaN,597,606,...,NaN,NaN,13483093.0,English,Metab. Clin. Exp.,Article,Final,NaN,Scopus,2-s2.0-84939404132
3,Kowlessar O.D.,6602166586;,MANAGEMENT OF MILD AND SEVERE ALCOHOLIC PANCRE...,1975,Annals of the New York Academy of Sciences,252,1,NaN,208,213,...,NaN,NaN,238450.0,English,Ann. New York Acad. Sci.,Article,Final,NaN,Scopus,2-s2.0-0016861433
4,Louria D.B.,7005869883;,Defenses against candida infections,1976,"International Journal of Radiation Oncology, B...",1,3-4,NaN,309,311,...,NaN,IOBPD,823139.0,English,Int J Radiat Oncol Biol Phys,Article,Final,NaN,Scopus,2-s2.0-0017099240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,"Montalvo M., Nallapaneni P., Hassan S., Nurko ...",55618980000;57396359900;57472321900;7003348777...,Autoimmune gastrointestinal dysmotility follow...,2022,Neurogastroenterology and Motility,34,7,e14314,NaN,NaN,...,NaN,NMOTE,34984765.0,English,Neurogastroenterol. Motil.,Article,Final,"All Open Access, Green",Scopus,2-s2.0-85122228589
982,"Ferrari T.C.A., Sampaio C.C.L., Ganzella C.G.,...",7003707979;57222582020;57779198600;57226058557...,Ascite chyleuse à Mycobacterium tuberculosis: ...,2022,Medecine Nucleaire,46,4,NaN,194,197,...,NaN,MNIME,NaN,English; French,Med. Nucl.,Article,Final,NaN,Scopus,2-s2.0-85133358045
983,"Little J.S., Shapiro R.M., Aleissa M.M., Kim A...",57216691364;57198153940;57217288754;5773750000...,Invasive Yeast Infection after Haploidentical ...,2022,Transplantation and Cellular Therapy,28,8,NaN,508.e1,508.e8,...,NaN,NaN,35526780.0,English,Transplant. Cell. Ther,Article,Final,NaN,Scopus,2-s2.0-85131814398
984,"Haeusler G.M., Lehrnbecher T., Agyeman P.K.A.,...",54389024100;7003278668;57735666000;57205628770...,Clostridioides difficile infection in paediatr...,2022,European Journal of Cancer,171,NaN,NaN,1,9,...,NaN,EJCAE,35696884.0,English,Eur. J. Cancer,Article,Final,NaN,Scopus,2-s2.0-85131718454


In [8]:
# import data
# data_df = pd.read_csv('E:\GIT_REPOS\LAB\Literature_summary\TPN\Papers\\scopus.csv')
data_df = data_df[data_df["Abstract"] != '[No abstract available]']
data_df.reset_index(inplace=True, drop=True)
data_df["Abstract"] = data_df["Abstract"].str.replace(r'(', '')
data_df["Abstract"] = data_df["Abstract"].str.replace(r')', '')
data_df["Abstract"] = data_df["Abstract"].str.replace(r"'", '')
data_df["Abstract"] = data_df["Abstract"].str.replace(r"'", '')
data_df["Abstract"] = data_df["Abstract"].str.replace(r'"', '')
data_df["Abstract"] = data_df["Abstract"].str.replace(r'"', '')
data_df["Abstract"] = data_df["Abstract"].astype(str)

C:\Users\GCM\AppData\Local\Temp\ipykernel_19780\412532311.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_df["Abstract"] = data_df["Abstract"].str.replace(r'(', '')
C:\Users\GCM\AppData\Local\Temp\ipykernel_19780\412532311.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["Abstract"] = data_df["Abstract"].str.replace(r'(', '')
C:\Users\GCM\AppData\Local\Temp\ipykernel_19780\412532311.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings w

In [9]:
win_size = 100
start_point = 0 # default to 0
# coref_lst = []
entities_df_lst = []
for i in range(start_point, len(data_df), win_size):
    coref_series = coref_res(text_series=data_df["Abstract"].iloc[i:i+win_size])
    print('coref done', i)
    link_entities_series = link_entities(text_series=coref_series)
    print('entity linking done', i)
    entities_df = pd.concat(link_entities_series.tolist())
    print('df create done', i)
    entities_df_lst.append(entities_df)
    print('df to list done', i, '\n')
all_entities_df = pd.concat(entities_df_lst)
all_entities_df.reset_index(drop=True, inplace=True)
edge_lst_df = all_entities_df.value_counts().reset_index().rename(columns={0: "count"})
edge_lst_df.to_csv('entity_weighted_edgelist.csv')

coref done 0
Could not extract relationships for text
entity linking done 0
df create done 0
df to list done 0 

coref done 100
Could not extract relationships for text
entity linking done 100
df create done 100
df to list done 100 

coref done 200
Could not extract relationships for text
entity linking done 200
df create done 200
df to list done 200 

coref done 300
Could not extract relationships for text
Could not extract relationships for text
Could not extract relationships for text
entity linking done 300
df create done 300
df to list done 300 

coref done 400
Could not extract relationships for text
entity linking done 400
df create done 400
df to list done 400 

coref done 500
entity linking done 500
df create done 500
df to list done 500 

coref done 600
Could not extract relationships for text
Could not extract relationships for text
entity linking done 600
df create done 600
df to list done 600 

coref done 700
entity linking done 700
df create done 700
df to list done 700 


In [8]:
# coref_series = coref_res(text_series=data_df["Abstract"][:1000])
# link_entities_series = link_entities(text_series=coref_series)
# all_entities_df = pd.concat(link_entities_series.tolist())